In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from qubex.simulator import Simulator, System, Transmon, Control

In [ ]:
transmon = Transmon(
    label="Q00",
    dimension=3,
    frequency=7.5,
    anharmonicity=-0.35,
)

system = System(
    transmons=[transmon],
)

simulator = Simulator(system)

initial_state = system.state({transmon.label: "0"})

In [ ]:
from qubex.pulse import DragGauss

duration = 30

alpha = 2 * np.pi * transmon.anharmonicity

pulse = DragGauss(
    duration=duration,
    amplitude=1,
    sigma=duration / 6,
    beta=-1 / alpha,
)
norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
pulse = pulse.scaled(norm_factor)
pulse.plot()
pulse.values

In [ ]:
steps_per_sample = 4

control = Control(
    target=transmon.label,
    frequency=transmon.frequency,
    waveform=pulse.values,
    steps_per_sample=steps_per_sample,
    sampling_period=pulse.SAMPLING_PERIOD,
)
control.values

In [ ]:
plt.plot(control.times, control.values.real)
plt.plot(control.times, control.values.imag)
plt.grid()

In [ ]:
import numpy as np

N = len(control.values)
dt = control.sampling_period / steps_per_sample

F_ = np.fft.fft(control.values)
f_ = np.fft.fftfreq(N, dt)
F = np.fft.fftshift(F_)
f = np.fft.fftshift(f_)

plt.plot(f, F.real, label="real")
plt.plot(f, F.imag, label="imag")
plt.plot(f, np.abs(F), label="abs")
plt.grid()
plt.legend()

In [ ]:
result = simulator.simulate(
    control=control,
    initial_state=initial_state,
)

In [ ]:
result.show_last_population(transmon.label)

In [ ]:
result.plot_population_dynamics()

In [ ]:
result.display_bloch_sphere(transmon.label)